In [1]:
data_var = '2024-02-18'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6926,2024-02-18,Alemanha Pro A,11:00,Bayreuth,PS Karlsruhe,1.76,1.97,165.5,1.80,1.86,-2.5,2.04,1.69,I7PBI8HC,0.568182,0.507614,0.555556,0.537634,0.075796,0.8,0.4,NaN,NaN,122.468,32.107537,0.262171,157.216,48.820705,0.310533,116.234,168.384,159.96,224.00,0.0,0.0,0.0,0.0,0.079621,0.023184,0.132701,0.76,0.152,5.000000,0.0,0.0,0.0,3.25,0.650,1.492308,0.0,0.0,0.0
6927,2024-02-18,Alemanha Pro A,11:00,Frankfurt,Giessen,1.54,2.37,158.5,1.80,1.86,-4.5,1.97,1.74,OpQ7Jl26,0.649351,0.421941,0.555556,0.537634,0.071292,0.4,0.4,NaN,NaN,106.084,14.090077,0.132820,126.280,26.353421,0.208690,105.514,161.680,96.60,121.94,0.0,0.0,0.0,0.0,0.300204,0.023184,0.087674,-2.59,-0.518,-1.042471,0.0,0.0,0.0,-0.60,-0.120,-11.416667,0.0,0.0,0.0
6928,2024-02-18,Alemanha Pro A,12:00,Dresden,Artland,1.41,2.76,167.5,1.86,1.84,-6.5,2.01,1.72,WATFHSWI,0.709220,0.362319,0.537634,0.543478,0.071539,0.6,0.8,NaN,NaN,155.514,32.372853,0.208167,298.146,163.693228,0.549037,126.960,298.594,109.50,266.22,0.0,0.0,0.0,0.0,0.457839,0.007644,0.109952,-3.86,-0.772,-0.531088,0.0,0.0,0.0,2.87,0.574,3.066202,0.0,0.0,0.0
6929,2024-02-18,Alemanha Dbbl Feminina,12:30,Leverkusen F,Halle F,3.31,1.30,140.5,1.85,1.85,6.5,2.00,1.72,tEIbkFzl,0.302115,0.769231,0.540541,0.540541,0.071346,0.0,0.0,NaN,NaN,359.958,281.285934,0.781441,222.910,119.858786,0.537700,0.000,0.000,142.14,401.52,0.0,0.0,0.0,0.0,0.616609,0.000000,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
6930,2024-02-18,Argentina Liga A,21:00,Quimsa,San Lorenzo,1.22,3.98,154.5,1.83,1.87,-10.5,1.96,1.75,GCtrh5oP,0.819672,0.251256,0.546448,0.534759,0.070928,0.6,0.4,NaN,NaN,123.812,27.597296,0.222897,183.702,80.018954,0.435591,109.982,193.632,102.00,256.31,0.0,0.0,0.0,0.0,0.750621,0.015289,0.080050,1.09,0.218,1.009174,0.0,0.0,0.0,-3.42,-0.684,-4.356725,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7067,2024-02-18,Mundo Amistoso Internacional,14:30,Siria,Iraque,3.39,1.27,144.5,1.81,1.84,4.5,1.83,1.87,0E3tcQ1t,0.294985,0.787402,0.552486,0.543478,0.082387,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.643376,0.011624,0.015289,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7068,2024-02-18,Noruega Blno,14:00,Bærum,Fyllingen,5.61,1.11,168.5,1.82,1.82,10.5,2.07,1.67,AVH78mSB,0.178253,0.900901,0.549451,0.549451,0.079154,0.2,0.6,NaN,NaN,139.648,47.077557,0.337116,808.982,1541.911734,1.905990,99.990,113.944,114.66,152.00,0.0,0.0,0.0,0.0,0.947018,0.000000,0.151253,-0.48,-0.096,-48.020833,0.0,0.0,0.0,-0.58,-0.116,-0.948276,0.0,0.0,0.0
7069,2024-02-18,Polônia Copa Da Polônia,13:30,Legia,Ostrow Wielkopolski,2.04,1.69,172.5,1.84,1.84,1.5,1.88,1.79,MRUpn7F5,0.490196,0.591716,0.543478,0.543478,0.081912,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,196.80,250.98,0.0,0.0,0.0,0.0,0.132701,0.000000,0.034681,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7070,2024-02-18,Romênia Copa Da Romênia,15:30,Cluj-Napoca,Rapid Bucuresti,1.03,9.45,157.5,1.82,1.83,-20.5,1.95,1.74,xIJ1DqE4,0.970874,0.105820,0.549451,0.546448,0.076694,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,97.52,0.00,0.0,0.0,0.0,0.0,1.136229,0.003875,0.080484,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
